<a href="https://colab.research.google.com/github/FrancescoCortopassi/copilot-codespaces-vscode/blob/main/Lezione_62_to_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEZIONE 62: Utilizzare la REGRESSIONE LINEARE nel trading algoritmico

In [79]:
#LEZIONE_62: In questo capitolo del corso si parla di: 1.Teoria della Regressione Lineare
#2. Predire il prezzo delle azioni con la regressione lineare
#3. Creare strategie di trading

In [80]:
#LEZIONE_63
#La prima cosa da sapere è che la Regressione Lineare è il più semplice algoritmo
#nel Machine Learning.

#La formula prevede che si sottragga IL VALORE REALE da un VALORE di PREVISONE



In [81]:
#LEZIONE_64: Importare i DATI
!pip install yfinance
!pip install ta

In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import ta

In [83]:
#importare i dati
df=pd.read_csv("/content/AMZN_H1_202001021600_202501312200.csv", delimiter="\t")
df.columns=["Date","Time","Open","High","Low","Close","Tickvol","Vol","Spread"]
df.index.name="Amazon Stocks"
df.head()
#IMPORTANTE: Quando si lavora con le STOCKS tramite Machine learning gli algoritmi di regressione
#lineare lavorano solo se la colonna di calcolo è indicizzata come CLOSE

,Date,Time,Open,High,Low,Close,Tickvol,Vol,Spread
Amazon Stocks,,,,,,,,,
0,2020.01.02,16:00:00,93.86,93.95,93.20,93.42,2921,0,0
1,2020.01.02,17:00:00,93.42,93.68,93.29,93.45,1951,0,0
2,2020.01.02,18:00:00,93.45,93.74,93.30,93.72,1637,0,0
3,2020.01.02,19:00:00,93.72,93.94,93.72,93.82,1493,0,0
4,2020.01.02,20:00:00,93.82,94.67,93.81,94.65,1773,0,0


In [84]:
#LEZIONE_65: FEATURE ENGINEERING del prezzo delle Amazon Stocks
#Richiamo la FUNZIONE creata nella lezione_60
def feature_engineering(df):
  #si copia il Dataframe per evitare interferenze nei dati
  df_copy= df.copy()
  #creo i "returns"
  df_copy["returns"]= df_copy['Close'].pct_change(1)
  #Creare le SMA's
  df_copy ["SMA 15"]= df_copy["Close"].rolling(15).mean().shift(1)
  df_copy ["SMA 60"]= df_copy["Close"].rolling(60).mean().shift(1)
  #Creare la volatilità MSD
  df_copy ["MSD 15"]= df_copy["returns"].rolling(15).std().shift(1)
  df_copy ["MSD 60"]= df_copy["returns"].rolling(60).std().shift(1)
  #Creare l'RSI
  df_copy["RSI"]= ta.momentum.RSIIndicator(df_copy["Close"], window=14, fillna= False).rsi()


  return df_copy


In [85]:
dfc= feature_engineering (df)
dfc

,Date,Time,Open,High,Low,Close,Tickvol,Vol,Spread,returns,SMA 15,SMA 60,MSD 15,MSD 60,RSI
Amazon Stocks,,,,,,,,,,,,,,,
0,2020.01.02,16:00:00,93.86,93.95,93.20,93.42,2921,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.01.02,17:00:00,93.42,93.68,93.29,93.45,1951,0,0,0.000321,NaN,NaN,NaN,NaN,NaN
2,2020.01.02,18:00:00,93.45,93.74,93.30,93.72,1637,0,0,0.002889,NaN,NaN,NaN,NaN,NaN
3,2020.01.02,19:00:00,93.72,93.94,93.72,93.82,1493,0,0,0.001067,NaN,NaN,NaN,NaN,NaN
4,2020.01.02,20:00:00,93.82,94.67,93.81,94.65,1773,0,0,0.008847,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8889,2025.01.31,18:00:00,239.29,239.80,238.47,239.57,5717,0,3,0.001170,236.668667,234.500500,0.007582,0.006947,59.898938
8890,2025.01.31,19:00:00,239.58,239.85,238.49,239.42,5605,0,3,-0.000626,236.749333,234.722833,0.007470,0.006931,59.332579
8891,2025.01.31,20:00:00,239.43,240.28,238.14,238.64,6250,0,3,-0.003258,236.876000,234.947167,0.007400,0.006929,56.348935


In [106]:
#LEZIONE_66: Suddivisione dei DATI (TRAINING SET/TEST SET)
#Per prima cosa dobbiamo comprendere in quale percentuale suddividere i dati
#Generalmente l'80% dei dati si utilizza per DDESTRARE il MODELLO
#Il 20% dei dati viene invece utilizzato per TESTARE IL MODELLO

#Percentuale del TRAINING SET
split= int(0.80*len(dfc))

#Creazione del TRAINING SET
X_train= dfc[["Close", "SMA 15", "SMA 60", "MSD 15", "MSD 60", "RSI"]].iloc[:split]
y_train=dfc[["returns"]].iloc[:split]

#Fare la stessa cosa con il TEST SET
X_test= dfc[["Close", "SMA 15", "SMA 60", "MSD 15", "MSD 60", "RSI"]].iloc[split:]
y_test=dfc[["returns"]].iloc[split:]

In [125]:
#Per comprendere meglio come splittare i dati   vado a visualizzare
#la x_train e la y_train
X_train

,Close,SMA 15,SMA 60,MSD 15,MSD 60,RSI
Amazon Stocks,,,,,,
0,93.42,NaN,NaN,NaN,NaN,NaN
1,93.45,NaN,NaN,NaN,NaN,NaN
2,93.72,NaN,NaN,NaN,NaN,NaN
3,93.82,NaN,NaN,NaN,NaN,NaN
4,94.65,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
7110,156.78,156.023333,154.222333,0.003319,0.004632,59.053036
7111,156.05,156.177333,154.282000,0.003294,0.004414,52.807529
7112,157.74,156.267333,154.310833,0.003597,0.004366,62.654673


In [88]:
#NOTA: Non riesco a capire come mai non mi calcola l'RSI. DA RIVEDERE!!!

In [126]:
y_train

,returns
Amazon Stocks,
0,NaN
1,0.000321
2,0.002889
3,0.001067
4,0.008847
...,...
7110,-0.002037
7111,-0.004656
7112,0.010830


In [90]:
#LEZIONE_67: ADDESTRAMENTO della REGRESSIONE LINEARE
#La prima cosa da fare è importare la Classe "Linear Regression"
#con la libreria scikitlearn

In [91]:
#Importare la Classe
import sklearn
from sklearn.linear_model import LinearRegression
#Inizializzare la Classe
reg= LinearRegression()
#Cancellare i dati mancanti NAN
X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]
#Fittare il modello
reg.fit (X_train,y_train)


LinearRegression()

In [128]:
#Creare la colonna PREDICTION all'interno del DataSet
#Per fare questo la prima cosa che dobbiamo fare è
#"concatenare" i dati che abbiamo splittato

X= np.concatenate((X_train,X_test), axis=0)

pd.DataFrame(X)

,0,1,2,3,4,5
0,93.42,NaN,NaN,NaN,NaN,NaN
1,93.45,NaN,NaN,NaN,NaN,NaN
2,93.72,NaN,NaN,NaN,NaN,NaN
3,93.82,NaN,NaN,NaN,NaN,NaN
4,94.65,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8889,239.57,236.668667,234.500500,0.007582,0.006947,59.898938
8890,239.42,236.749333,234.722833,0.007470,0.006931,59.332579
8891,238.64,236.876000,234.947167,0.007400,0.006929,56.348935
8892,238.11,236.946667,235.120833,0.007465,0.006847,54.349021


LEZIONE DA TERMINARE: Purtroppo sono bloccato con il codice ed in questo momento preferisco andare avanti con il corso e ritornare su questa parte in un secondo momento.
DA REVISIONARE!!!!